In [ ]:
import pandas as pd
import numpy as np
import os

print("--- ЭТАП 1: Загрузка и первичная обработка ---")

# --- 1.1: Определяем пути к файлам (замени на свои, если нужно) ---
# Убедись, что файлы лежат там, откуда Colab может их прочитать
# (например, загружены в сессию или лежат на подключенном Google Drive)
fires_file = 'fires.csv'
supplies_file = 'supplies.csv'
temperature_file = 'temperature.csv'
weather_files = ['weather_data_2019.csv', 'weather_data_2020.csv'] # Файлы погоды

# --- 1.2: Обработка fires.csv ---
print(f"\nОбработка {fires_file}...")
df_fires_a1 = pd.DataFrame() # Инициализируем пустым на случай ошибки
try:
    if not os.path.exists(fires_file):
        raise FileNotFoundError(f"Файл {fires_file} не найден.")
    df_fires = pd.read_csv(fires_file)
    print(f" Исходный размер: {df_fires.shape}")
    df_fires_a1 = df_fires[df_fires['Груз'] == 'A1'].copy()
    print(f" Размер после фильтрации по 'A1': {df_fires_a1.shape}")
except FileNotFoundError as fnf_error:
    print(f" Ошибка: {fnf_error}")
except Exception as e:
    print(f" Ошибка при обработке {fires_file}: {e}")

# --- 1.3: Обработка supplies.csv ---
print(f"\nОбработка {supplies_file}...")
df_supplies_a1 = pd.DataFrame()
try:
    if not os.path.exists(supplies_file):
        raise FileNotFoundError(f"Файл {supplies_file} не найден.")
    df_supplies = pd.read_csv(supplies_file)
    print(f" Исходный размер: {df_supplies.shape}")
    df_supplies_a1 = df_supplies[df_supplies['Наим. ЕТСНГ'] == 'A1'].copy()
    print(f" Размер после фильтрации по 'A1': {df_supplies_a1.shape}")
except FileNotFoundError as fnf_error:
    print(f" Ошибка: {fnf_error}")
except Exception as e:
    print(f" Ошибка при обработке {supplies_file}: {e}")

# --- 1.4: Обработка temperature.csv ---
print(f"\nОбработка {temperature_file}...")
df_temp_a1 = pd.DataFrame()
try:
    if not os.path.exists(temperature_file):
        raise FileNotFoundError(f"Файл {temperature_file} не найден.")
    df_temp = pd.read_csv(temperature_file)
    print(f" Исходный размер: {df_temp.shape}")
    cols_to_drop_temp = ['Пикет', 'Смена']
    existing_cols_temp = [col for col in cols_to_drop_temp if col in df_temp.columns]
    if existing_cols_temp:
        df_temp.drop(columns=existing_cols_temp, inplace=True)
        print(f" Удалены колонки: {existing_cols_temp}")
    df_temp['Марка'] = df_temp['Марка'].astype(str).apply(lambda x: 'A1' if x.strip().startswith('A1') else x)
    print(" Марка нормализована.")
    df_temp_a1 = df_temp[df_temp['Марка'] == 'A1'].copy()
    print(f" Размер после фильтрации по 'A1': {df_temp_a1.shape}")
except FileNotFoundError as fnf_error:
    print(f" Ошибка: {fnf_error}")
except Exception as e:
    print(f" Ошибка при обработке {temperature_file}: {e}")

# --- 1.5: Обработка и объединение файлов погоды ---
print(f"\nОбработка файлов погоды: {weather_files}...")
all_weather_dfs = []
df_weather_combined = pd.DataFrame()
cols_to_drop_weather = ['wind_dir', 'v_max', 'v_avg']
for weather_file in weather_files:
    print(f" Обработка файла: {weather_file}")
    try:
        if not os.path.exists(weather_file):
            raise FileNotFoundError(f"Файл {weather_file} не найден.")
        df_weather_single = pd.read_csv(weather_file)
        print(f"  Исходный размер: {df_weather_single.shape}")
        existing_cols_weather = [col for col in cols_to_drop_weather if col in df_weather_single.columns]
        if existing_cols_weather:
            df_weather_single.drop(columns=existing_cols_weather, inplace=True)
            print(f"  Удалены колонки: {existing_cols_weather}")
        all_weather_dfs.append(df_weather_single)
    except FileNotFoundError as fnf_error:
        print(f"  Ошибка: {fnf_error}. Файл пропущен.")
    except Exception as e:
        print(f"  Ошибка при обработке {weather_file}: {e}. Файл пропущен.")

if all_weather_dfs:
    df_weather_combined = pd.concat(all_weather_dfs, ignore_index=True)
    print(f"\nДанные погоды за {len(all_weather_dfs)} год(а) объединены.")
    print(f" Общий размер данных погоды: {df_weather_combined.shape}")
else:
    print("\nНе удалось загрузить ни одного файла погоды. DataFrame погоды пуст.")

print("\n--- ЭТАП 1 ЗАВЕРШЕН ---")
print("-" * 50)

# --- ЭТАП 2: Преобразование типов и анализ пропусков ---
print("\n--- ЭТАП 2: Преобразование типов и анализ пропусков ---")

# Функция для безопасного преобразования в datetime
def safe_to_datetime(series, **kwargs):
    converted = pd.to_datetime(series, errors='coerce', **kwargs)
    return converted

# Функция для безопасного преобразования в числовой тип
def safe_to_numeric(series):
    if pd.api.types.is_string_dtype(series):
        if series.str.contains(',').any():
             # print(f"  Замена ',' на '.' в столбце '{series.name}'") # Раскомментируй для отладки
             series = series.str.replace(',', '.', regex=False)
    return pd.to_numeric(series, errors='coerce')

# 2.1 Обработка df_fires_a1
print("\nОбработка типов данных в df_fires_a1...")
if not df_fires_a1.empty:
    date_cols_fires = ['Дата составления', 'Дата начала', 'Дата оконч.', 'Нач. форм. штабеля']
    for col in date_cols_fires:
        if col in df_fires_a1.columns and not pd.api.types.is_datetime64_any_dtype(df_fires_a1[col]):
            print(f" Преобразование '{col}' в datetime...")
            df_fires_a1[col] = safe_to_datetime(df_fires_a1[col])
    num_cols_fires = ['Вес по акту, тн']
    for col in num_cols_fires:
        if col in df_fires_a1.columns and not pd.api.types.is_numeric_dtype(df_fires_a1[col]):
            print(f" Преобразование '{col}' в numeric...")
            df_fires_a1[col] = safe_to_numeric(df_fires_a1[col])
    print("Типы данных df_fires_a1 после преобразования:")
    df_fires_a1.info()
else:
    print("DataFrame df_fires_a1 пуст или не был создан на этапе 1.")

# 2.2 Обработка df_supplies_a1
print("\nОбработка типов данных в df_supplies_a1...")
if not df_supplies_a1.empty:
    date_cols_supplies = ['ВыгрузкаНаСклад', 'ПогрузкаНаСудно']
    for col in date_cols_supplies:
         if col in df_supplies_a1.columns and not pd.api.types.is_datetime64_any_dtype(df_supplies_a1[col]):
            print(f" Преобразование '{col}' в datetime...")
            df_supplies_a1[col] = safe_to_datetime(df_supplies_a1[col])
    num_cols_supplies = ['На склад, тн', 'На судно, тн']
    for col in num_cols_supplies:
        if col in df_supplies_a1.columns and not pd.api.types.is_numeric_dtype(df_supplies_a1[col]):
            print(f" Преобразование '{col}' в numeric...")
            df_supplies_a1[col] = safe_to_numeric(df_supplies_a1[col])
    print("\nТипы данных df_supplies_a1 после преобразования:")
    df_supplies_a1.info()
else:
    print("DataFrame df_supplies_a1 пуст или не был создан на этапе 1.")

# 2.3 Обработка df_temp_a1
print("\nОбработка типов данных в df_temp_a1...")
if not df_temp_a1.empty:
    date_cols_temp = ['Дата акта']
    for col in date_cols_temp:
        if col in df_temp_a1.columns and not pd.api.types.is_datetime64_any_dtype(df_temp_a1[col]):
            print(f" Преобразование '{col}' в datetime...")
            df_temp_a1[col] = safe_to_datetime(df_temp_a1[col])
    num_cols_temp = ['Максимальная температура']
    for col in num_cols_temp:
         if col in df_temp_a1.columns and not pd.api.types.is_numeric_dtype(df_temp_a1[col]):
            print(f" Преобразование '{col}' в numeric...")
            df_temp_a1[col] = safe_to_numeric(df_temp_a1[col])
    print("\nТипы данных df_temp_a1 после преобразования:")
    df_temp_a1.info()
else:
    print("DataFrame df_temp_a1 пуст или не был создан на этапе 1.")

# 2.4 Обработка df_weather_combined
print("\nОбработка типов данных в df_weather_combined...")
if not df_weather_combined.empty:
    date_cols_weather = ['date']
    for col in date_cols_weather:
        if col in df_weather_combined.columns and not pd.api.types.is_datetime64_any_dtype(df_weather_combined[col]):
            print(f" Преобразование '{col}' в datetime...")
            df_weather_combined[col] = safe_to_datetime(df_weather_combined[col])
    num_cols_weather = ['t', 'p', 'humidity', 'precipitation', 'cloudcover', 'visibility', 'weather_code']
    for col in num_cols_weather:
        if col in df_weather_combined.columns and not pd.api.types.is_numeric_dtype(df_weather_combined[col]):
             print(f" Преобразование '{col}' в numeric...")
             df_weather_combined[col] = safe_to_numeric(df_weather_combined[col])
    print("\nТипы данных df_weather_combined после преобразования:")
    df_weather_combined.info()
else:
    print("DataFrame df_weather_combined пуст или не был создан на этапе 1.")

print("-" * 30)

# --- 2.5 Анализ пропусков ---
print("\n--- Анализ пропущенных значений ---")

def analyze_missing_values(df, df_name):
    if df is None or df.empty:
        print(f"\nDataFrame '{df_name}' пуст или не существует, анализ пропусков невозможен.")
        return
    print(f"\nАнализ пропусков для '{df_name}':")
    missing_count = df.isnull().sum()
    missing_percent = (df.isnull().mean() * 100).round(2)
    missing_df = pd.DataFrame({'Количество пропусков': missing_count, 'Процент пропусков (%)': missing_percent})
    missing_df = missing_df[missing_df['Количество пропусков'] > 0]
    if not missing_df.empty:
        print(missing_df.sort_values(by='Количество пропусков', ascending=False))
    else:
        print("Пропущенные значения отсутствуют.")

analyze_missing_values(df_fires_a1 if 'df_fires_a1' in locals() else None, "df_fires_a1")
analyze_missing_values(df_supplies_a1 if 'df_supplies_a1' in locals() else None, "df_supplies_a1")
analyze_missing_values(df_temp_a1 if 'df_temp_a1' in locals() else None, "df_temp_a1")
analyze_missing_values(df_weather_combined if 'df_weather_combined' in locals() else None, "df_weather_combined")

print("\n--- ЭТАП 2 ЗАВЕРШЕН ---")
print("-" * 50)

# --- ЭТАП 3: Сохранение (опционально) ---
# Если нужно сохранить DataFrames с правильными типами для дальнейшего использования
save_typed_files = True # Поставь True для сохранения

if save_typed_files:
    print("\n--- ЭТАП 3: Сохранение файлов с преобразованными типами ---")
    output_dir_typed = "processed_data_typed"
    if not os.path.exists(output_dir_typed):
        os.makedirs(output_dir_typed)
        print(f"Создана директория: {output_dir_typed}")
    try:
        if 'df_fires_a1' in locals() and not df_fires_a1.empty:
            df_fires_a1.to_csv(os.path.join(output_dir_typed,'fires_A1_typed.csv'), index=False)
        if 'df_supplies_a1' in locals() and not df_supplies_a1.empty:
            df_supplies_a1.to_csv(os.path.join(output_dir_typed,'supplies_A1_typed.csv'), index=False)
        if 'df_temp_a1' in locals() and not df_temp_a1.empty:
            df_temp_a1.to_csv(os.path.join(output_dir_typed,'temperature_A1_typed.csv'), index=False)
        if 'df_weather_combined' in locals() and not df_weather_combined.empty:
            df_weather_combined.to_csv(os.path.join(output_dir_typed,'weather_2019_2020_typed.csv'), index=False)
        print(f"Файлы сохранены в папку: {output_dir_typed}")
    except Exception as e:
        print(f"Ошибка при сохранении файлов: {e}")

--- ЭТАП 1: Загрузка и первичная обработка ---

Обработка fires.csv...
 Ошибка: Файл fires.csv не найден.

Обработка supplies.csv...
 Исходный размер: (6323, 7)
 Размер после фильтрации по 'A1': (4285, 7)

Обработка temperature.csv...
 Исходный размер: (4106, 7)
 Удалены колонки: ['Пикет', 'Смена']
 Марка нормализована.
 Размер после фильтрации по 'A1': (4095, 5)

Обработка файлов погоды: ['weather_data_2019.csv', 'weather_data_2020.csv']...
 Обработка файла: weather_data_2019.csv
  Ошибка: Файл weather_data_2019.csv не найден.. Файл пропущен.
 Обработка файла: weather_data_2020.csv
  Исходный размер: (8760, 11)
  Удалены колонки: ['wind_dir', 'v_max', 'v_avg']

Данные погоды за 1 год(а) объединены.
 Общий размер данных погоды: (8760, 8)

--- ЭТАП 1 ЗАВЕРШЕН ---
--------------------------------------------------

--- ЭТАП 2: Преобразование типов и анализ пропусков ---

Обработка типов данных в df_fires_a1...
DataFrame df_fires_a1 пуст или не был создан на этапе 1.

Обработка типов данн

In [ ]:
# --- Шаг 3: Агрегация данных о погоде до дневного уровня ---
print("\n--- Шаг 3: Агрегация данных о погоде до дневного уровня ---")

if 'df_weather_combined' in locals() and not df_weather_combined.empty:
    if 'date' in df_weather_combined.columns and pd.api.types.is_datetime64_any_dtype(df_weather_combined['date']):

        # Убедимся, что 'date' - это индекс для удобной ресемплинг/агрегации
        # Если дата еще не индекс:
        if not isinstance(df_weather_combined.index, pd.DatetimeIndex):
             print(" Установка 'date' в качестве индекса...")
             # Сортируем по дате перед установкой индекса (важно для некоторых агрегаций)
             df_weather_combined.sort_values('date', inplace=True)
             df_weather_combined.set_index('date', inplace=True)

        # Создаем столбец с датой без времени для группировки
        df_weather_combined['day_date'] = df_weather_combined.index.date

        print(" Агрегация почасовых данных по дням...")
        # Определяем функции агрегации для каждого столбца
        agg_functions = {
            't': ['mean', 'min', 'max'], # Средняя, минимальная, максимальная температура за день
            'p': ['mean', 'min', 'max'], # Давление
            'humidity': ['mean', 'min'],  # Влажность (макс обычно 100, мин важнее)
            'precipitation': 'sum',       # Сумма осадков за день
            'cloudcover': 'mean',         # Средняя облачность
            # 'weather_code': ['median', 'last'] # Медианный код погоды или последний за день (можно выбрать)
            'weather_code': lambda x: x.mode()[0] if not x.mode().empty else np.nan # Самый частый код погоды за день
        }

        # Убедимся, что все колонки для агрегации существуют
        valid_agg_functions = {k: v for k, v in agg_functions.items() if k in df_weather_combined.columns}
        print(f" Колонки для агрегации: {list(valid_agg_functions.keys())}")

        if valid_agg_functions:
            # Группируем по дню и применяем агрегацию
            df_weather_daily = df_weather_combined.groupby('day_date').agg(valid_agg_functions)

            # Переименовываем столбцы для понятности (например, t_mean, t_min, ...)
            df_weather_daily.columns = ['_'.join(col).strip() for col in df_weather_daily.columns.values]
            # Исправляем имя для 'precipitation_sum' и 'weather_code_<lambda>'
            df_weather_daily.rename(columns={'precipitation_sum': 'precip_total_day',
                                             'weather_code_<lambda>':'weather_code_mode'}, inplace=True)


            # Преобразуем индекс (который сейчас 'day_date' типа object) обратно в datetime
            df_weather_daily.index = pd.to_datetime(df_weather_daily.index)

            print("\nПример агрегированных данных погоды (df_weather_daily):")
            print(df_weather_daily.head())
            print("\nРазмер агрегированных данных:", df_weather_daily.shape)
            df_weather_daily.info()

        else:
            print(" Ошибка: Не найдены колонки для агрегации в df_weather_combined.")
            df_weather_daily = pd.DataFrame() # Создаем пустой на случай ошибки

        # Удаляем вспомогательный столбец из исходного df_weather_combined (если он больше не нужен)
        # df_weather_combined.drop(columns=['day_date'], inplace=True, errors='ignore')
        # Можно вернуть индекс обратно в столбец, если нужно
        # df_weather_combined.reset_index(inplace=True)

    else:
        print(" Ошибка: Столбец 'date' не найден или не является datetime в df_weather_combined.")
        df_weather_daily = pd.DataFrame()
else:
    print(" DataFrame df_weather_combined не найден или пуст. Агрегация погоды пропущена.")
    df_weather_daily = pd.DataFrame()

print("\n--- Шаг 3 ЗАВЕРШЕН ---")
print("-" * 50)


--- Шаг 3: Агрегация данных о погоде до дневного уровня ---
 Установка 'date' в качестве индекса...
 Агрегация почасовых данных по дням...
 Колонки для агрегации: ['t', 'p', 'humidity', 'precipitation', 'cloudcover', 'weather_code']

Пример агрегированных данных погоды (df_weather_daily):
              t_mean  t_min  t_max       p_mean   p_min   p_max  \
day_date                                                          
2020-01-01  6.129167    4.9    8.7  1014.779167  1011.5  1020.7   
2020-01-02  3.983333    2.9    4.7  1025.512500  1021.2  1028.2   
2020-01-03  5.200000    3.1    8.0  1023.545833  1019.9  1027.2   
2020-01-04  7.933333    6.9    8.6  1016.208333  1013.7  1019.4   
2020-01-05  8.400000    7.3    9.0  1013.220833  1012.0  1015.6   

            humidity_mean  humidity_min  precip_total_day  cloudcover_mean  \
day_date                                                                     
2020-01-01      80.458333            72               6.7        81.958333   
2020-

In [ ]:
import pandas as pd

print("\n--- Шаг 4: Создание Мастер-сетки (Склад-Штабель-День) ---")

# Проверяем наличие необходимых DataFrame'ов
if ('df_supplies_a1' not in locals() or df_supplies_a1.empty) and \
   ('df_temp_a1' not in locals() or df_temp_a1.empty):
    print("Ошибка: Не найдены DataFrame'ы df_supplies_a1 и df_temp_a1 для определения уникальных штабелей.")
    master_grid = pd.DataFrame() # Создаем пустой DataFrame
else:
    # 4.1 Определяем временной диапазон (Январь 2019 - Декабрь 2020)
    start_date = pd.Timestamp('2019-01-01')
    end_date = pd.Timestamp('2020-12-31')
    # Создаем полный диапазон дат с шагом в 1 день
    all_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    print(f"Временной диапазон: с {start_date.date()} по {end_date.date()}")
    print(f"Всего дней в диапазоне: {len(all_dates)}")

    # 4.2 Находим уникальные комбинации Склад/Штабель
    unique_stacks = set()

    # Из df_supplies_a1
    if 'df_supplies_a1' in locals() and not df_supplies_a1.empty:
        # Убедимся, что типы совместимы (например, оба строки или оба числа)
        # Преобразуем в строки для надежности сравнения и объединения
        stacks_supplies = df_supplies_a1[['Склад', 'Штабель']].astype(str).drop_duplicates()
        unique_stacks.update(list(map(tuple, stacks_supplies.values))) # Добавляем как кортежи
        print(f" Уникальных пар (Склад, Штабель) найдено в df_supplies_a1: {len(stacks_supplies)}")

    # Из df_temp_a1
    if 'df_temp_a1' in locals() and not df_temp_a1.empty:
        stacks_temp = df_temp_a1[['Склад', 'Штабель']].astype(str).drop_duplicates()
        unique_stacks.update(list(map(tuple, stacks_temp.values))) # Добавляем как кортежи
        print(f" Уникальных пар (Склад, Штабель) найдено в df_temp_a1: {len(stacks_temp)}")

    # Из df_fires_a1 (на всякий случай, если какой-то горевший штабель не попал в другие таблицы)
    if 'df_fires_a1' in locals() and not df_fires_a1.empty:
        stacks_fires = df_fires_a1[['Склад', 'Штабель']].astype(str).drop_duplicates()
        unique_stacks.update(list(map(tuple, stacks_fires.values)))
        print(f" Уникальных пар (Склад, Штабель) найдено в df_fires_a1: {len(stacks_fires)}")


    # Преобразуем set обратно в DataFrame
    if unique_stacks:
        unique_stacks_df = pd.DataFrame(list(unique_stacks), columns=['Склад', 'Штабель'])
        # Опционально: можно преобразовать обратно в числовые типы, если нужно
        # unique_stacks_df['Склад'] = pd.to_numeric(unique_stacks_df['Склад'], errors='ignore')
        # unique_stacks_df['Штабель'] = pd.to_numeric(unique_stacks_df['Штабель'], errors='ignore')

        print(f"\nОбщее количество уникальных пар (Склад, Штабель): {len(unique_stacks_df)}")
        # print("Пример уникальных пар:")
        # print(unique_stacks_df.head())

        # 4.3 Создаем полную сетку (декартово произведение дат и штабелей)
        print("\nСоздание мастер-сетки (декартово произведение)...")
        # Создаем DataFrame с датами
        dates_df = pd.DataFrame({'Дата': all_dates})

        # Используем cross join (декартово произведение)
        master_grid = unique_stacks_df.merge(dates_df, how='cross')

        print(f"Размер созданной мастер-сетки: {master_grid.shape}")
        print("(Ожидаемое количество строк: {} уникальных штабелей * {} дней = {})".format(
            len(unique_stacks_df), len(all_dates), len(unique_stacks_df) * len(all_dates)
        ))
        print("\nПример мастер-сетки (master_grid):")
        print(master_grid.head())
        master_grid.info()

    else:
        print("Ошибка: Не удалось найти ни одной уникальной пары (Склад, Штабель). Мастер-сетка не создана.")
        master_grid = pd.DataFrame()


print("\n--- Шаг 4 ЗАВЕРШЕН ---")
print("-" * 50)


--- Шаг 4: Создание Мастер-сетки (Склад-Штабель-День) ---
Временной диапазон: с 2019-01-01 по 2020-12-31
Всего дней в диапазоне: 731
 Уникальных пар (Склад, Штабель) найдено в df_supplies_a1: 93
 Уникальных пар (Склад, Штабель) найдено в df_temp_a1: 69

Общее количество уникальных пар (Склад, Штабель): 96

Создание мастер-сетки (декартово произведение)...
Размер созданной мастер-сетки: (70176, 3)
(Ожидаемое количество строк: 96 уникальных штабелей * 731 дней = 70176)

Пример мастер-сетки (master_grid):
  Склад Штабель       Дата
0     3       7 2019-01-01
1     3       7 2019-01-02
2     3       7 2019-01-03
3     3       7 2019-01-04
4     3       7 2019-01-05
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70176 entries, 0 to 70175
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Склад    70176 non-null  object        
 1   Штабель  70176 non-null  object        
 2   Дата     70176 non-null  datetime64[

In [ ]:
# --- Шаг 4.Б: Сохранение Мастер-сетки ---
print("\n--- Шаг 4.Б: Сохранение Мастер-сетки ---")

save_master_grid_file = True # Поставь False, если не хочешь сохранять

if save_master_grid_file:
    if 'master_grid' in locals() and not master_grid.empty:
        output_dir_grid = "processed_data_final" # Папка для промежуточных/финальных данных
        if not os.path.exists(output_dir_grid):
            os.makedirs(output_dir_grid)
            print(f"Создана директория: {output_dir_grid}")

        output_path = os.path.join(output_dir_grid, 'master_grid_sklad_shtabel_den.csv')
        try:
            master_grid.to_csv(output_path, index=False)
            print(f"Мастер-сетка успешно сохранена в файл: {output_path}")
        except Exception as e:
            print(f"Ошибка при сохранении мастер-сетки: {e}")
    else:
        print("Мастер-сетка (master_grid) не найдена или пуста. Файл не сохранен.")
else:
    print("Сохранение мастер-сетки пропущено (save_master_grid_file = False).")

print("\n--- Шаг 4.Б ЗАВЕРШЕН ---")
print("-" * 50)


--- Шаг 4.Б: Сохранение Мастер-сетки ---
Создана директория: processed_data_final
Мастер-сетка успешно сохранена в файл: processed_data_final/master_grid_sklad_shtabel_den.csv

--- Шаг 4.Б ЗАВЕРШЕН ---
--------------------------------------------------


In [ ]:
print("\n--- Шаг 5: Присоединение агрегированных данных о погоде к Мастер-сетке ---")

# Проверяем наличие необходимых DataFrame'ов
if 'master_grid' in locals() and not master_grid.empty and \
   'df_weather_daily' in locals() and not df_weather_daily.empty:

    # Убедимся, что 'Дата' в master_grid это datetime
    if not pd.api.types.is_datetime64_any_dtype(master_grid['Дата']):
        print(" Предупреждение: Преобразование 'Дата' в master_grid в datetime...")
        master_grid['Дата'] = pd.to_datetime(master_grid['Дата'], errors='coerce')

    # Убедимся, что индекс в df_weather_daily это datetime
    if not isinstance(df_weather_daily.index, pd.DatetimeIndex):
         print(" Предупреждение: Преобразование индекса df_weather_daily в datetime...")
         try:
             df_weather_daily.index = pd.to_datetime(df_weather_daily.index, errors='coerce')
             # Проверим на NaT после преобразования
             if df_weather_daily.index.isnull().any():
                 print(" Ошибка: В индексе df_weather_daily появились NaT после преобразования. Слияние невозможно.")
                 # Можно добавить код для удаления строк с NaT или остановки
             else:
                  print(" Индекс df_weather_daily успешно преобразован.")
         except Exception as e:
             print(f" Ошибка при преобразовании индекса df_weather_daily: {e}. Слияние невозможно.")
             # Устанавливаем флаг ошибки или выходим
             can_merge = False
    else:
        can_merge = True # Индекс уже datetime

    # Проверяем, что нет NaT в ключевых столбцах перед слиянием
    if master_grid['Дата'].isnull().any():
        print(" Предупреждение: В столбце 'Дата' мастер-сетки есть пропуски (NaT). Строки с пропусками не получат данных о погоде.")
        # Решение: либо удалить эти строки, либо оставить как есть

    if can_merge:
        print("\nВыполнение слияния (merge) master_grid с df_weather_daily...")
        # Используем левое слияние: сохраняем все строки из master_grid
        # Сливаем по 'Дата' из master_grid и по индексу из df_weather_daily
        master_grid_with_weather = pd.merge(
            master_grid,
            df_weather_daily,
            left_on='Дата',    # Ключ в левом DataFrame (master_grid)
            right_index=True, # Использовать индекс как ключ в правом DataFrame (df_weather_daily)
            how='left'        # Тип слияния: сохранить все строки из master_grid
        )

        print(f"\nРазмер сетки до слияния: {master_grid.shape}")
        print(f"Размер сетки после слияния с погодой: {master_grid_with_weather.shape}")

        # Проверяем, появились ли новые столбцы
        added_cols = set(master_grid_with_weather.columns) - set(master_grid.columns)
        print(f"Добавленные столбцы погоды: {list(added_cols)}")

        # Заменяем старую master_grid на обновленную
        master_grid = master_grid_with_weather
        print("\nМастер-сетка обновлена данными о погоде.")

        print("\nПример обновленной мастер-сетки (первые строки):")
        print(master_grid.head())
        print("\nИнформация об обновленной мастер-сетке:")
        master_grid.info()

        # Проверим процент пропусков в добавленных колонках (ожидаемо, если погода не за все дни)
        print("\nПроцент пропусков в добавленных колонках погоды:")
        missing_weather_pct = master_grid[list(added_cols)].isnull().mean() * 100
        print(missing_weather_pct.round(2))


else:
    print("Ошибка: Один или оба DataFrame'а (master_grid, df_weather_daily) не найдены или пусты. Слияние невозможно.")


print("\n--- Шаг 5 ЗАВЕРШЕН ---")
print("-" * 50)


--- Шаг 5: Присоединение агрегированных данных о погоде к Мастер-сетке ---

Выполнение слияния (merge) master_grid с df_weather_daily...

Размер сетки до слияния: (70176, 3)
Размер сетки после слияния с погодой: (70176, 14)
Добавленные столбцы погоды: ['p_min', 'weather_code_mode', 't_mean', 't_min', 'p_max', 'p_mean', 'humidity_min', 'precip_total_day', 't_max', 'cloudcover_mean', 'humidity_mean']

Мастер-сетка обновлена данными о погоде.

Пример обновленной мастер-сетки (первые строки):
  Склад Штабель       Дата  t_mean  t_min  t_max  p_mean  p_min  p_max  \
0     3       7 2019-01-01     NaN    NaN    NaN     NaN    NaN    NaN   
1     3       7 2019-01-02     NaN    NaN    NaN     NaN    NaN    NaN   
2     3       7 2019-01-03     NaN    NaN    NaN     NaN    NaN    NaN   
3     3       7 2019-01-04     NaN    NaN    NaN     NaN    NaN    NaN   
4     3       7 2019-01-05     NaN    NaN    NaN     NaN    NaN    NaN   

   humidity_mean  humidity_min  precip_total_day  cloudcover

In [ ]:
print("\n--- Шаг 6: Присоединение данных о температуре штабеля ---")

# Проверяем наличие необходимых DataFrame'ов
if 'master_grid' in locals() and not master_grid.empty and \
   'df_temp_a1' in locals() and not df_temp_a1.empty:

    # 6.1 Подготовка df_temp_a1 для слияния
    print("Подготовка данных о температуре (df_temp_a1)...")
    temp_to_merge = df_temp_a1[['Склад', 'Штабель', 'Дата акта', 'Максимальная температура']].copy()

    # Убедимся, что 'Дата акта' это datetime
    if not pd.api.types.is_datetime64_any_dtype(temp_to_merge['Дата акта']):
        print(" Предупреждение: Преобразование 'Дата акта' в datetime...")
        temp_to_merge['Дата акта'] = pd.to_datetime(temp_to_merge['Дата акта'], errors='coerce')

    # Извлекаем только дату (без времени) для слияния с дневной сеткой
    # Используем dt.normalize() для обнуления времени, сохраняя тип datetime64[ns]
    temp_to_merge['Дата'] = temp_to_merge['Дата акта'].dt.normalize()

    # Преобразуем Склад/Штабель в строки для консистентности ключей слияния
    temp_to_merge['Склад'] = temp_to_merge['Склад'].astype(str)
    temp_to_merge['Штабель'] = temp_to_merge['Штабель'].astype(str)

    # Обработка дубликатов: Если за один день было несколько замеров для одного штабеля,
    # берем МАКСИМАЛЬНУЮ температуру за этот день.
    print(" Агрегация температуры: взятие max() для дубликатов Склад/Штабель/Дата...")
    temp_aggregated = temp_to_merge.groupby(['Склад', 'Штабель', 'Дата'], as_index=False)['Максимальная температура'].max()
    print(f" Размер данных температуры после агрегации: {temp_aggregated.shape}")

    # 6.2 Подготовка master_grid (проверка типов ключей)
    if not pd.api.types.is_string_dtype(master_grid['Склад']):
         print(" Предупреждение: Преобразование 'Склад' в master_grid в строку...")
         master_grid['Склад'] = master_grid['Склад'].astype(str)
    if not pd.api.types.is_string_dtype(master_grid['Штабель']):
         print(" Предупреждение: Преобразование 'Штабель' в master_grid в строку...")
         master_grid['Штабель'] = master_grid['Штабель'].astype(str)
    if not pd.api.types.is_datetime64_any_dtype(master_grid['Дата']):
         print(" Предупреждение: Преобразование 'Дата' в master_grid в datetime...")
         master_grid['Дата'] = pd.to_datetime(master_grid['Дата'], errors='coerce')

    # Удаляем строки с NaT в датах перед слиянием, если они есть
    initial_rows = len(master_grid)
    master_grid.dropna(subset=['Дата'], inplace=True)
    if len(master_grid) < initial_rows:
        print(f" Предупреждение: Удалено {initial_rows - len(master_grid)} строк с NaT в 'Дата' из master_grid.")

    initial_rows_temp = len(temp_aggregated)
    temp_aggregated.dropna(subset=['Дата', 'Склад', 'Штабель', 'Максимальная температура'], inplace=True)
    if len(temp_aggregated) < initial_rows_temp:
         print(f" Предупреждение: Удалено {initial_rows_temp - len(temp_aggregated)} строк с NaN/NaT из temp_aggregated.")


    # 6.3 Выполнение слияния
    print("\nВыполнение слияния (merge) master_grid с temp_aggregated...")
    master_grid_with_temp = pd.merge(
        master_grid,
        temp_aggregated[['Склад', 'Штабель', 'Дата', 'Максимальная температура']],
        on=['Склад', 'Штабель', 'Дата'], # Ключи слияния
        how='left'                      # Сохраняем все строки из master_grid
    )

    print(f"\nРазмер сетки до слияния: {master_grid.shape}")
    print(f"Размер сетки после слияния с температурой: {master_grid_with_temp.shape}")

    # Переименуем добавленную колонку для ясности
    master_grid_with_temp.rename(columns={'Максимальная температура': 'Temp_Measure_Max'}, inplace=True)
    print("Добавлен столбец 'Temp_Measure_Max'")

    # Заменяем старую master_grid на обновленную
    master_grid = master_grid_with_temp
    print("\nМастер-сетка обновлена данными о температуре.")

    print("\nПример обновленной мастер-сетки (первые строки):")
    print(master_grid.head())
    # print("\nИнформация об обновленной мастер-сетке:")
    # master_grid.info() # Может быть очень длинным, выведем только пропуски

    # Проверим процент пропусков в добавленной колонке температуры
    # Ожидается высокий процент, так как замеры не ежедневные
    print("\nПроцент пропусков в добавленном столбце 'Temp_Measure_Max':")
    missing_temp_pct = master_grid['Temp_Measure_Max'].isnull().mean() * 100
    print(f"{missing_temp_pct:.2f}%")

else:
    print("Ошибка: Один или оба DataFrame'а (master_grid, df_temp_a1) не найдены или пусты. Слияние невозможно.")

print("\n--- Шаг 6 ЗАВЕРШЕН ---")
print("-" * 50)


--- Шаг 6: Присоединение данных о температуре штабеля ---
Подготовка данных о температуре (df_temp_a1)...
 Агрегация температуры: взятие max() для дубликатов Склад/Штабель/Дата...
 Размер данных температуры после агрегации: (2175, 4)

Выполнение слияния (merge) master_grid с temp_aggregated...

Размер сетки до слияния: (70176, 14)
Размер сетки после слияния с температурой: (70176, 15)
Добавлен столбец 'Temp_Measure_Max'

Мастер-сетка обновлена данными о температуре.

Пример обновленной мастер-сетки (первые строки):
  Склад Штабель       Дата  t_mean  t_min  t_max  p_mean  p_min  p_max  \
0     3       7 2019-01-01     NaN    NaN    NaN     NaN    NaN    NaN   
1     3       7 2019-01-02     NaN    NaN    NaN     NaN    NaN    NaN   
2     3       7 2019-01-03     NaN    NaN    NaN     NaN    NaN    NaN   
3     3       7 2019-01-04     NaN    NaN    NaN     NaN    NaN    NaN   
4     3       7 2019-01-05     NaN    NaN    NaN     NaN    NaN    NaN   

   humidity_mean  humidity_min  p

In [ ]:
import pandas as pd
import numpy as np

print("\n--- Шаг 7: Присоединение данных о поступлении/отгрузке угля (ОТЛАДОЧНАЯ ВЕРСИЯ) ---")

# Проверяем наличие необходимых DataFrame'ов
if 'master_grid' not in locals() or master_grid.empty or \
   'df_supplies_a1' not in locals() or df_supplies_a1.empty:
      print("Ошибка: Отсутствует master_grid или df_supplies_a1. Шаг 7 не может быть выполнен.")
else:
    # 7.1 Подготовка df_supplies_a1
    print("Подготовка данных о поставках (df_supplies_a1)...")
    supplies_to_agg = df_supplies_a1[['Склад', 'Штабель', 'ВыгрузкаНаСклад', 'ПогрузкаНаСудно', 'На склад, тн', 'На судно, тн']].copy()
    print(f" Исходный размер supplies_to_agg: {supplies_to_agg.shape}")

    # Преобразования типов (добавим проверки)
    try:
        if not pd.api.types.is_datetime64_any_dtype(supplies_to_agg['ВыгрузкаНаСклад']):
            supplies_to_agg['ВыгрузкаНаСклад'] = pd.to_datetime(supplies_to_agg['ВыгрузкаНаСклад'], errors='coerce')
        if not pd.api.types.is_datetime64_any_dtype(supplies_to_agg['ПогрузкаНаСудно']):
            supplies_to_agg['ПогрузкаНаСудно'] = pd.to_datetime(supplies_to_agg['ПогрузкаНаСудно'], errors='coerce')
        if not pd.api.types.is_numeric_dtype(supplies_to_agg['На склад, тн']):
            supplies_to_agg['На склад, тн'] = pd.to_numeric(supplies_to_agg['На склад, тн'], errors='coerce')
        if not pd.api.types.is_numeric_dtype(supplies_to_agg['На судно, тн']):
            supplies_to_agg['На судно, тн'] = pd.to_numeric(supplies_to_agg['На судно, тн'], errors='coerce')
        print(" Типы данных преобразованы (или уже были корректны).")
    except Exception as e:
        print(f" ОШИБКА при преобразовании типов в supplies_to_agg: {e}")

    # Заполнение NaN нулями
    supplies_to_agg['На склад, тн'] = supplies_to_agg['На склад, тн'].fillna(0)
    supplies_to_agg['На судно, тн'] = supplies_to_agg['На судно, тн'].fillna(0)
    print(" NaN в тоннаже заменены на 0.")

    # --- Агрегация поступлений ---
    print("\nАгрегация поступлений...")
    arrivals = supplies_to_agg[supplies_to_agg['На склад, тн'] > 0].copy()
    print(f" Найдено строк с поступлениями (arrivals): {len(arrivals)}")
    if not arrivals.empty:
        arrivals['Дата'] = arrivals['ВыгрузкаНаСклад'].dt.normalize()
        # --- ОТЛАДКА ---
        print(f" Пример дат в arrivals['Дата']: {arrivals['Дата'].head().tolist()}")
        print(f" Пропуски в arrivals['Дата']: {arrivals['Дата'].isnull().sum()}")
        arrivals.dropna(subset=['Дата'], inplace=True) # Удалим строки с некорректной датой выгрузки
        print(f" Строк arrivals после удаления NaN в дате: {len(arrivals)}")
        # --- КОНЕЦ ОТЛАДКИ ---
        arrivals_agg = arrivals.groupby(['Склад', 'Штабель', 'Дата'], as_index=False)['На склад, тн'].sum()
        arrivals_agg.rename(columns={'На склад, тн': 'Added_Today_Tons'}, inplace=True)
        arrivals_agg['Склад'] = arrivals_agg['Склад'].astype(str)
        arrivals_agg['Штабель'] = arrivals_agg['Штабель'].astype(str)
        print(f" Размер arrivals_agg (поступления по дням): {arrivals_agg.shape}")
        # --- ОТЛАДКА ---
        print(" Пример arrivals_agg:")
        print(arrivals_agg.head())
        # --- КОНЕЦ ОТЛАДКИ ---
    else:
        arrivals_agg = pd.DataFrame(columns=['Склад', 'Штабель', 'Дата', 'Added_Today_Tons'])

    # --- Агрегация отгрузок ---
    print("\nАгрегация отгрузок...")
    shipments = supplies_to_agg[supplies_to_agg['На судно, тн'] > 0].copy()
    print(f" Найдено строк с отгрузками (shipments): {len(shipments)}")
    if not shipments.empty:
        shipments['Дата'] = shipments['ПогрузкаНаСудно'].dt.normalize()
         # --- ОТЛАДКА ---
        print(f" Пример дат в shipments['Дата']: {shipments['Дата'].head().tolist()}")
        print(f" Пропуски в shipments['Дата']: {shipments['Дата'].isnull().sum()}")
        shipments.dropna(subset=['Дата'], inplace=True) # Удалим строки с некорректной датой погрузки
        print(f" Строк shipments после удаления NaN в дате: {len(shipments)}")
        # --- КОНЕЦ ОТЛАДКИ ---
        shipments_agg = shipments.groupby(['Склад', 'Штабель', 'Дата'], as_index=False)['На судно, тн'].sum()
        shipments_agg.rename(columns={'На судно, тн': 'Removed_Today_Tons'}, inplace=True)
        shipments_agg['Склад'] = shipments_agg['Склад'].astype(str)
        shipments_agg['Штабель'] = shipments_agg['Штабель'].astype(str)
        print(f" Размер shipments_agg (отгрузки по дням): {shipments_agg.shape}")
         # --- ОТЛАДКА ---
        print(" Пример shipments_agg:")
        print(shipments_agg.head())
        # --- КОНЕЦ ОТЛАДКИ ---
    else:
        shipments_agg = pd.DataFrame(columns=['Склад', 'Штабель', 'Дата', 'Removed_Today_Tons'])

    # 7.2 Подготовка master_grid
    print("\nПодготовка master_grid к слиянию...")
    if not pd.api.types.is_string_dtype(master_grid['Склад']):
         master_grid['Склад'] = master_grid['Склад'].astype(str)
    if not pd.api.types.is_string_dtype(master_grid['Штабель']):
         master_grid['Штабель'] = master_grid['Штабель'].astype(str)
    if not pd.api.types.is_datetime64_any_dtype(master_grid['Дата']):
         master_grid['Дата'] = pd.to_datetime(master_grid['Дата'], errors='coerce')
    initial_rows = len(master_grid)
    master_grid.dropna(subset=['Дата'], inplace=True)
    if len(master_grid) < initial_rows:
         print(f" Удалено {initial_rows - len(master_grid)} строк с NaT в 'Дата' из master_grid.")
    print(" Типы ключей в master_grid проверены/преобразованы.")
     # --- ОТЛАДКА ---
    print(f" Типы ключей в master_grid: Склад={master_grid['Склад'].dtype}, Штабель={master_grid['Штабель'].dtype}, Дата={master_grid['Дата'].dtype}")
    print(f" Типы ключей в arrivals_agg: Склад={arrivals_agg['Склад'].dtype if not arrivals_agg.empty else 'N/A'}, Штабель={arrivals_agg['Штабель'].dtype if not arrivals_agg.empty else 'N/A'}, Дата={arrivals_agg['Дата'].dtype if not arrivals_agg.empty else 'N/A'}")
    print(f" Типы ключей в shipments_agg: Склад={shipments_agg['Склад'].dtype if not shipments_agg.empty else 'N/A'}, Штабель={shipments_agg['Штабель'].dtype if not shipments_agg.empty else 'N/A'}, Дата={shipments_agg['Дата'].dtype if not shipments_agg.empty else 'N/A'}")
     # --- КОНЕЦ ОТЛАДКИ ---


    # 7.3 Выполнение слияния
    print("\nВыполнение слияния master_grid с arrivals_agg...")
    master_grid = pd.merge(
        master_grid,
        arrivals_agg[['Склад', 'Штабель', 'Дата', 'Added_Today_Tons']],
        on=['Склад', 'Штабель', 'Дата'],
        how='left'
    )
     # --- ОТЛАДКА ---
    print(f" Колонки в master_grid ПОСЛЕ слияния с arrivals_agg: {master_grid.columns.tolist()}")
    print(f" Проверка NaN в Added_Today_Tons СРАЗУ ПОСЛЕ слияния: {master_grid['Added_Today_Tons'].isnull().sum() if 'Added_Today_Tons' in master_grid.columns else 'Колонка НЕ добавлена!'}")
     # --- КОНЕЦ ОТЛАДКИ ---

    print("\nВыполнение слияния master_grid с shipments_agg...")
    master_grid = pd.merge(
        master_grid,
        shipments_agg[['Склад', 'Штабель', 'Дата', 'Removed_Today_Tons']],
        on=['Склад', 'Штабель', 'Дата'],
        how='left'
    )
     # --- ОТЛАДКА ---
    print(f" Колонки в master_grid ПОСЛЕ слияния с shipments_agg: {master_grid.columns.tolist()}")
    print(f" Проверка NaN в Removed_Today_Tons СРАЗУ ПОСЛЕ слияния: {master_grid['Removed_Today_Tons'].isnull().sum() if 'Removed_Today_Tons' in master_grid.columns else 'Колонка НЕ добавлена!'}")
     # --- КОНЕЦ ОТЛАДКИ ---

    # 7.4 Заполнение NaN нулями
    print("\nЗаполнение NaN нулями в 'Added_Today_Tons' и 'Removed_Today_Tons'...")
    if 'Added_Today_Tons' in master_grid.columns:
        master_grid['Added_Today_Tons'] = master_grid['Added_Today_Tons'].fillna(0)
        print(" NaN в Added_Today_Tons заменены на 0.")
    else:
        print(" ПРЕДУПРЕЖДЕНИЕ: Колонка Added_Today_Tons не найдена для заполнения NaN!")

    if 'Removed_Today_Tons' in master_grid.columns:
        master_grid['Removed_Today_Tons'] = master_grid['Removed_Today_Tons'].fillna(0)
        print(" NaN в Removed_Today_Tons заменены на 0.")
    else:
        print(" ПРЕДУПРЕЖДЕНИЕ: Колонка Removed_Today_Tons не найдена для заполнения NaN!")

    print("\nМастер-сетка обновлена данными о поступлениях/отгрузках.")
    print("\nФинальный набор колонок в master_grid:", master_grid.columns.tolist())
    print("\nПример финальной мастер-сетки (Шаг 7):")
    print(master_grid.head())

print("\n--- Шаг 7 (ОТЛАДОЧНАЯ ВЕРСИЯ) ЗАВЕРШЕН ---")
print("-" * 50)


--- Шаг 7: Присоединение данных о поступлении/отгрузке угля (ОТЛАДОЧНАЯ ВЕРСИЯ) ---
Подготовка данных о поставках (df_supplies_a1)...
 Исходный размер supplies_to_agg: (4285, 6)
 Типы данных преобразованы (или уже были корректны).
 NaN в тоннаже заменены на 0.

Агрегация поступлений...
 Найдено строк с поступлениями (arrivals): 4285
 Пример дат в arrivals['Дата']: [Timestamp('2019-01-02 00:00:00'), Timestamp('2019-01-02 00:00:00'), Timestamp('2019-01-02 00:00:00'), Timestamp('2019-01-03 00:00:00'), Timestamp('2019-01-03 00:00:00')]
 Пропуски в arrivals['Дата']: 0
 Строк arrivals после удаления NaN в дате: 4285
 Размер arrivals_agg (поступления по дням): (3260, 4)
 Пример arrivals_agg:
  Склад Штабель       Дата  Added_Today_Tons
0     3       1 2019-07-18        41129.4515
1     3       1 2019-07-19        17861.4665
2     3       1 2019-07-20        24048.4975
3     3       1 2019-07-21          749.8105
4     3       1 2019-07-22          199.4695

Агрегация отгрузок...
 Найдено стр

In [ ]:
import pandas as pd
import numpy as np
import os

print("\n--- Шаг 8: Расчет динамических признаков (Вес, Возраст, Дни с замера) ---")

# Проверяем наличие master_grid
if 'master_grid' not in locals() or master_grid.empty:
    print("Ошибка: DataFrame 'master_grid' не найден или пуст. Расчет невозможен.")
else:
    # --- 8.1 Расчет текущего веса ---
    print("Расчет текущего оценочного веса штабеля ('Current_Weight_Tons')...")

    # Убедимся, что необходимые колонки существуют и числовые
    if 'Added_Today_Tons' in master_grid.columns and \
       'Removed_Today_Tons' in master_grid.columns and \
       pd.api.types.is_numeric_dtype(master_grid['Added_Today_Tons']) and \
       pd.api.types.is_numeric_dtype(master_grid['Removed_Today_Tons']):

        # Рассчитываем дневное изменение веса
        master_grid['Daily_Weight_Change'] = master_grid['Added_Today_Tons'] - master_grid['Removed_Today_Tons']

        # Сортируем данные - КРИТИЧЕСКИ ВАЖНО для кумулятивных расчетов!
        print(" Сортировка данных по Склад, Штабель, Дата...")
        master_grid.sort_values(by=['Склад', 'Штабель', 'Дата'], inplace=True)

        # Рассчитываем кумулятивную сумму изменения веса в рамках каждого штабеля
        print(" Расчет кумулятивной суммы веса...")
        master_grid['Current_Weight_Tons'] = master_grid.groupby(['Склад', 'Штабель'])['Daily_Weight_Change'].cumsum()

        # Проверка на отрицательный вес (может указывать на проблемы с данными)
        neg_weight_count = (master_grid['Current_Weight_Tons'] < -0.01).sum() # Небольшой допуск на ошибки округления
        if neg_weight_count > 0:
            print(f" Предупреждение: Обнаружено {neg_weight_count} строк с отрицательным расчетным весом (< -0.01).")
            # По желанию можно установить минимальный вес 0:
            # master_grid['Current_Weight_Tons'] = master_grid['Current_Weight_Tons'].clip(lower=0)
            # print(" Отрицательный вес был заменен на 0.")
    else:
        print(" Ошибка: Колонки 'Added_Today_Tons' или 'Removed_Today_Tons' отсутствуют или не являются числовыми.")
        master_grid['Current_Weight_Tons'] = np.nan # Добавляем колонку с NaN

    # --- 8.2 Расчет возраста штабеля ---
    print("\nРасчет возраста штабеля ('Stack_Age_Days')...")
    # Используем дату первой выгрузки на склад как дату формирования (см. обсуждение ограничений)
    if 'df_supplies_a1' in locals() and not df_supplies_a1.empty:
        print(" Поиск даты первого поступления для каждого штабеля...")
        arrivals_data = df_supplies_a1[df_supplies_a1['На склад, тн'] > 0].copy()
        if 'ВыгрузкаНаСклад' in arrivals_data.columns:
             # Убедимся что дата - datetime
             if not pd.api.types.is_datetime64_any_dtype(arrivals_data['ВыгрузкаНаСклад']):
                  arrivals_data['ВыгрузкаНаСклад'] = pd.to_datetime(arrivals_data['ВыгрузкаНаСклад'], errors='coerce')

             # Находим минимальную дату для каждой пары Склад/Штабель
             formation_dates = arrivals_data.loc[arrivals_data['ВыгрузкаНаСклад'].notna()].groupby(['Склад', 'Штабель'])['ВыгрузкаНаСклад'].min().reset_index()
             formation_dates.rename(columns={'ВыгрузкаНаСклад': 'Formation_Date'}, inplace=True)

             # Приводим ключи к строкам для слияния
             formation_dates['Склад'] = formation_dates['Склад'].astype(str)
             formation_dates['Штабель'] = formation_dates['Штабель'].astype(str)
             print(f" Найдено дат формирования: {len(formation_dates)}")

             # Присоединяем дату формирования к master_grid
             print(" Присоединение даты формирования...")
             master_grid = pd.merge(
                 master_grid,
                 formation_dates,
                 on=['Склад', 'Штабель'],
                 how='left'
             )

             # Рассчитываем возраст в днях
             if 'Formation_Date' in master_grid.columns:
                 print(" Расчет возраста в днях...")
                 # Убедимся, что обе колонки - datetime
                 if not pd.api.types.is_datetime64_any_dtype(master_grid['Дата']):
                     master_grid['Дата'] = pd.to_datetime(master_grid['Дата'], errors='coerce')
                 if not pd.api.types.is_datetime64_any_dtype(master_grid['Formation_Date']):
                      master_grid['Formation_Date'] = pd.to_datetime(master_grid['Formation_Date'], errors='coerce')

                 # Расчет разницы
                 time_diff = master_grid['Дата'] - master_grid['Formation_Date']
                 # Берем только количество дней, обрабатываем случаи до даты формирования или NaT
                 master_grid['Stack_Age_Days'] = time_diff.dt.days
                 master_grid.loc[master_grid['Stack_Age_Days'] < 0, 'Stack_Age_Days'] = 0 # Возраст не может быть < 0
                 # Пропуски в возрасте возникнут, если не было даты формирования

                 # Удаляем вспомогательную колонку даты формирования (если не нужна)
                 # master_grid.drop(columns=['Formation_Date'], inplace=True)

             else:
                 print(" Ошибка: Не удалось присоединить 'Formation_Date'.")
                 master_grid['Stack_Age_Days'] = np.nan

        else:
             print(" Ошибка: Отсутствует колонка 'ВыгрузкаНаСклад' в данных о поступлениях.")
             master_grid['Stack_Age_Days'] = np.nan
    else:
        print(" Предупреждение: DataFrame df_supplies_a1 не найден. Возраст штабеля не рассчитан.")
        master_grid['Stack_Age_Days'] = np.nan

    # --- 8.3 Расчет дней с последнего замера температуры ---
    print("\nРасчет дней с последнего замера температуры ('Days_Since_Last_Measurement')...")
    if 'Temp_Measure_Max' in master_grid.columns:
        # Создаем колонку с датой только там, где был замер
        master_grid['Measure_Date_If_Present'] = master_grid['Дата'].where(master_grid['Temp_Measure_Max'].notna())

        # Сортировка (уже должна быть, но для надежности)
        master_grid.sort_values(by=['Склад', 'Штабель', 'Дата'], inplace=True)

        # Заполняем пропуски датой последнего замера (forward fill) внутри каждой группы
        print(" Применение forward fill для даты последнего замера...")
        master_grid['Last_Known_Measure_Date'] = master_grid.groupby(['Склад', 'Штабель'])['Measure_Date_If_Present'].ffill()

        # Рассчитываем разницу в днях
        if 'Last_Known_Measure_Date' in master_grid.columns:
             # Убедимся, что Last_Known_Measure_Date - datetime
             if not pd.api.types.is_datetime64_any_dtype(master_grid['Last_Known_Measure_Date']):
                  master_grid['Last_Known_Measure_Date'] = pd.to_datetime(master_grid['Last_Known_Measure_Date'], errors='coerce')

             time_diff_measure = master_grid['Дата'] - master_grid['Last_Known_Measure_Date']
             master_grid['Days_Since_Last_Measurement'] = time_diff_measure.dt.days
             # Пропуски в результате будут там, где еще не было ни одного замера для штабеля

             # Удаляем вспомогательные колонки
             master_grid.drop(columns=['Measure_Date_If_Present', 'Last_Known_Measure_Date'], inplace=True, errors='ignore')
        else:
             print(" Ошибка: Не удалось создать 'Last_Known_Measure_Date'.")
             master_grid['Days_Since_Last_Measurement'] = np.nan

    else:
        print(" Предупреждение: Столбец 'Temp_Measure_Max' отсутствует. Расчет дней с замера невозможен.")
        master_grid['Days_Since_Last_Measurement'] = np.nan


    # --- 8.4 Вывод информации ---
    print("\nРасчет динамических признаков завершен.")
    print("Добавленные/обновленные колонки: 'Daily_Weight_Change', 'Current_Weight_Tons', 'Stack_Age_Days', 'Days_Since_Last_Measurement'")

    print("\nПример обновленной мастер-сетки:")
    print(master_grid.head())

    print("\nПроверка пропусков в новых колонках:")
    new_cols = ['Current_Weight_Tons', 'Stack_Age_Days', 'Days_Since_Last_Measurement']
    for col in new_cols:
        if col in master_grid.columns:
            missing_pct = master_grid[col].isnull().mean() * 100
            print(f" {col}: {missing_pct:.2f}% пропусков")
        else:
            print(f" {col}: столбец не найден")


    # --- 8.5 Сохранение результата ---
    print("\n--- Сохранение мастер-сетки с динамическими признаками ---")
    output_dir_final = "processed_data_final"
    if not os.path.exists(output_dir_final):
        os.makedirs(output_dir_final)
        print(f"Создана директория: {output_dir_final}")

    output_path_dynamic = os.path.join(output_dir_final, 'master_grid_with_dynamic_features.csv')
    try:
        master_grid.to_csv(output_path_dynamic, index=False)
        print(f"Мастер-сетка с динамическими признаками успешно сохранена в файл: {output_path_dynamic}")
    except Exception as e:
        print(f"Ошибка при сохранении файла: {e}")


print("\n--- Шаг 8 ЗАВЕРШЕН ---")
print("-" * 50)


--- Шаг 8: Расчет динамических признаков (Вес, Возраст, Дни с замера) ---
Расчет текущего оценочного веса штабеля ('Current_Weight_Tons')...
 Сортировка данных по Склад, Штабель, Дата...
 Расчет кумулятивной суммы веса...
 Предупреждение: Обнаружено 650 строк с отрицательным расчетным весом (< -0.01).

Расчет возраста штабеля ('Stack_Age_Days')...
 Поиск даты первого поступления для каждого штабеля...
 Найдено дат формирования: 93
 Присоединение даты формирования...
 Расчет возраста в днях...

Расчет дней с последнего замера температуры ('Days_Since_Last_Measurement')...
 Применение forward fill для даты последнего замера...

Расчет динамических признаков завершен.
Добавленные/обновленные колонки: 'Daily_Weight_Change', 'Current_Weight_Tons', 'Stack_Age_Days', 'Days_Since_Last_Measurement'

Пример обновленной мастер-сетки:
  Склад Штабель       Дата  t_mean  t_min  t_max  p_mean  p_min  p_max  \
0     3       0 2019-01-01     NaN    NaN    NaN     NaN    NaN    NaN   
1     3       0 

In [ ]:
import pandas as pd
import numpy as np
import os

print("\n--- Шаг 9: Добавление целевой переменной (Fire_Started_Today) ---")

# Проверяем наличие необходимых DataFrame'ов
if 'master_grid' not in locals() or master_grid.empty or \
   'df_fires_a1' not in locals() or df_fires_a1.empty:
      print("Ошибка: Отсутствует master_grid или df_fires_a1. Добавление целевой переменной невозможно.")
else:
    # 9.1 Подготовка df_fires_a1
    print("Подготовка данных о возгораниях (df_fires_a1)...")
    fires_to_merge = df_fires_a1[['Склад', 'Штабель', 'Дата начала']].copy()

    # Убедимся, что 'Дата начала' - datetime
    if not pd.api.types.is_datetime64_any_dtype(fires_to_merge['Дата начала']):
        fires_to_merge['Дата начала'] = pd.to_datetime(fires_to_merge['Дата начала'], errors='coerce')

    # Извлекаем только дату (без времени)
    fires_to_merge['Дата'] = fires_to_merge['Дата начала'].dt.normalize()

    # Приводим ключи к строкам
    fires_to_merge['Склад'] = fires_to_merge['Склад'].astype(str)
    fires_to_merge['Штабель'] = fires_to_merge['Штабель'].astype(str)

    # Удаляем строки с некорректной датой начала
    initial_fires_rows = len(fires_to_merge)
    fires_to_merge.dropna(subset=['Склад', 'Штабель', 'Дата'], inplace=True)
    if len(fires_to_merge) < initial_fires_rows:
         print(f" Удалено {initial_fires_rows - len(fires_to_merge)} строк с NaN в ключах/дате из данных о пожарах.")

    # Создаем колонку-индикатор пожара
    fires_to_merge['Fire_Started_Today'] = 1
    print(f" Найдено записей о начале пожаров: {len(fires_to_merge)}")

    # Обработка дубликатов (если вдруг в один день для одного штабеля >1 записи о начале пожара)
    # Оставляем только одну запись (max() на индикаторе даст 1)
    fires_agg = fires_to_merge.groupby(['Склад', 'Штабель', 'Дата'], as_index=False)['Fire_Started_Today'].max()
    print(f" Уникальных событий начала пожара (Склад/Штабель/День): {len(fires_agg)}")

    # 9.2 Подготовка master_grid (типы ключей уже должны быть проверены)
    print("\nПодготовка master_grid к слиянию (проверка типов ключей)...")
    if not pd.api.types.is_string_dtype(master_grid['Склад']): master_grid['Склад'] = master_grid['Склад'].astype(str)
    if not pd.api.types.is_string_dtype(master_grid['Штабель']): master_grid['Штабель'] = master_grid['Штабель'].astype(str)
    if not pd.api.types.is_datetime64_any_dtype(master_grid['Дата']): master_grid['Дата'] = pd.to_datetime(master_grid['Дата'], errors='coerce')
    master_grid.dropna(subset=['Дата'], inplace=True) # На всякий случай
    print(" Типы ключей в master_grid проверены.")


    # 9.3 Слияние с данными о пожарах
    print("\nВыполнение слияния master_grid с данными о пожарах...")
    master_grid = pd.merge(
        master_grid,
        fires_agg[['Склад', 'Штабель', 'Дата', 'Fire_Started_Today']],
        on=['Склад', 'Штабель', 'Дата'],
        how='left'
    )
    print(f" Размер сетки после слияния с данными о пожарах: {master_grid.shape}")
    print(f" Колонки в master_grid: {master_grid.columns.tolist()}")


    # 9.4 Заполнение NaN нулями в целевой переменной
    print("\nЗаполнение NaN в 'Fire_Started_Today' нулями...")
    if 'Fire_Started_Today' in master_grid.columns:
        master_grid['Fire_Started_Today'] = master_grid['Fire_Started_Today'].fillna(0).astype(int) # Заполняем 0 и преобразуем в int
        print(" NaN в Fire_Started_Today заменены на 0.")
        # Проверим, сколько всего пожаров в итоговой сетке
        total_fires_in_grid = master_grid['Fire_Started_Today'].sum()
        print(f" Общее количество дней с началом пожара в мастер-сетке: {total_fires_in_grid}")
        # Сравним с количеством событий в fires_agg - должно совпадать
        if total_fires_in_grid == len(fires_agg):
            print(" Количество совпадает с исходными агрегированными данными о пожарах. ОК.")
        else:
            print(f" ПРЕДУПРЕЖДЕНИЕ: Количество пожаров не совпадает! В сетке: {total_fires_in_grid}, в исходных: {len(fires_agg)}")
    else:
        print(" ОШИБКА: Колонка 'Fire_Started_Today' не найдена после слияния!")

    # 9.5 Сохранение результата (опционально, но полезно)
    save_final_grid = True
    if save_final_grid:
        print("\n--- Сохранение финальной мастер-сетки с целевой переменной ---")
        output_dir_final = "processed_data_final"
        if not os.path.exists(output_dir_final):
            os.makedirs(output_dir_final)
        output_path_final = os.path.join(output_dir_final, 'master_grid_final_with_target.csv')
        try:
            master_grid.to_csv(output_path_final, index=False)
            print(f"Финальная мастер-сетка успешно сохранена в файл: {output_path_final}")
        except Exception as e:
            print(f"Ошибка при сохранении файла: {e}")

print("\n--- Шаг 9 ЗАВЕРШЕН ---")
print("-" * 50)


--- Шаг 9: Добавление целевой переменной (Fire_Started_Today) ---
Ошибка: Отсутствует master_grid или df_fires_a1. Добавление целевой переменной невозможно.

--- Шаг 9 ЗАВЕРШЕН ---
--------------------------------------------------


In [ ]:
import pandas as pd
import numpy as np
import os

# --- Откат к состоянию после Шага 9 ---
print("--- Откат к состоянию DataFrame после Шага 9 ---")
print("(Включая погоду, температуру, поставки, базовые динамические признаки и ОРИГИНАЛЬНУЮ бинарную цель)")

output_dir_final = "processed_data_final"
# Имя файла, сохраненного в конце Шага 9
input_file_step9 = os.path.join(output_dir_final, 'master_grid_with_dynamic_features.csv')

# Попытка загрузить
if os.path.exists(input_file_step9):
    print(f"Загрузка данных из файла: {input_file_step9}...")
    try:
        # Загружаем данные, парсим дату
        df_reverted = pd.read_csv(input_file_step9, parse_dates=['Дата'])

        # Переименовываем DataFrame для дальнейшей работы (например, обратно в df_eda)
        df_eda = df_reverted.copy()
        print("Данные успешно загружены и скопированы в df_eda.")
        print(f"Размер DataFrame: {df_eda.shape}")
        print("\nКолонки DataFrame:")
        print(df_eda.columns.tolist())
        print("\nИнформация о типах данных:")
        df_eda.info()
        print("\nПример данных:")
        print(df_eda.head())
        print("\nПроверка целевой переменной (должна быть 0/1):")
        print(df_eda['Fire_Started_Today'].value_counts())

    except Exception as e:
        print(f"Ошибка при загрузке или обработке файла {input_file_step9}: {e}")
        print("Не удалось откатиться к нужному состоянию.")
        # Оставляем df_eda как есть или обнуляем, в зависимости от ситуации
        # df_eda = pd.DataFrame() # Раскомментируй, если нужно обнулить в случае ошибки
else:
    print(f"Ошибка: Файл {input_file_step9} не найден.")
    print("Не удалось откатиться к нужному состоянию.")
    print("Убедитесь, что Шаг 9 был выполнен и файл был сохранен с этим именем.")
    # Возможно, нужно будет перезапустить предыдущие шаги для генерации этого файла

print("\n--- Откат завершен (или не удался) ---")
print("-" * 50)

--- Откат к состоянию DataFrame после Шага 9 ---
(Включая погоду, температуру, поставки, базовые динамические признаки и ОРИГИНАЛЬНУЮ бинарную цель)
Загрузка данных из файла: processed_data_final/master_grid_with_dynamic_features.csv...
Данные успешно загружены и скопированы в df_eda.
Размер DataFrame: (70176, 22)

Колонки DataFrame:
['Склад', 'Штабель', 'Дата', 't_mean', 't_min', 't_max', 'p_mean', 'p_min', 'p_max', 'humidity_mean', 'humidity_min', 'precip_total_day', 'cloudcover_mean', 'weather_code_mode', 'Temp_Measure_Max', 'Added_Today_Tons', 'Removed_Today_Tons', 'Daily_Weight_Change', 'Current_Weight_Tons', 'Formation_Date', 'Stack_Age_Days', 'Days_Since_Last_Measurement']

Информация о типах данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70176 entries, 0 to 70175
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Склад                        70176 non-

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm

print("\n--- Шаг 11 (Альтернативный): Фильтрация DataFrame по замерам температуры и периодам перед пожарами ---")

# --- 11.1 Загрузка необходимых данных ---

# Загрузка основного DataFrame (состояние после Шага 9)
output_dir_final = "processed_data_final"
input_file_main = os.path.join(output_dir_final, 'master_grid_with_dynamic_features.csv') # Файл с бинарной целью
print(f"Загрузка основного DataFrame из {input_file_main}...")
if os.path.exists(input_file_main):
    df_main = pd.read_csv(input_file_main, parse_dates=['Дата'])
    print(f"Основной DataFrame загружен. Размер: {df_main.shape}")
else:
    print(f"Ошибка: Файл {input_file_main} не найден. Невозможно продолжить.")
    raise FileNotFoundError(f"Файл {input_file_main} не найден.")

# Загрузка данных о температуре (обработанных ранее)
input_file_temp = os.path.join("processed_data_typed", 'temperature_A1_typed.csv') # Файл с обработанными типами
# ИЛИ если сохраняли на другом шаге: os.path.join(output_dir_final, 'temperature_A1_processed.csv')
print(f"Загрузка данных о температуре из {input_file_temp}...")
if os.path.exists(input_file_temp):
    df_temp_source = pd.read_csv(input_file_temp, parse_dates=['Дата акта']) # Убедимся, что Дата акта парсится
    print(f"Данные о температуре загружены. Размер: {df_temp_source.shape}")
else:
    # Попробуем другой возможный путь
    input_file_temp_alt = os.path.join("processed_data", 'temperature_A1_processed.csv')
    if os.path.exists(input_file_temp_alt):
         print(f"Загрузка данных о температуре из альтернативного пути: {input_file_temp_alt}...")
         df_temp_source = pd.read_csv(input_file_temp_alt)
         # Преобразуем дату акта, если нужно
         if 'Дата акта' in df_temp_source.columns and not pd.api.types.is_datetime64_any_dtype(df_temp_source['Дата акта']):
              df_temp_source['Дата акта'] = pd.to_datetime(df_temp_source['Дата акта'], errors='coerce')
         print(f"Данные о температуре загружены. Размер: {df_temp_source.shape}")
    else:
        print(f"Ошибка: Файл данных о температуре не найден ни по одному из путей ({input_file_temp}, {input_file_temp_alt}). Невозможно продолжить.")
        raise FileNotFoundError("Файл данных о температуре не найден.")


# --- 11.2 Подготовка ключей и типов ---
print("\nПодготовка ключей и типов данных...")

# Преобразуем ключи в строки для надежности сравнения/объединения
df_main['Склад'] = df_main['Склад'].astype(str)
df_main['Штабель'] = df_main['Штабель'].astype(str)
df_temp_source['Склад'] = df_temp_source['Склад'].astype(str)
df_temp_source['Штабель'] = df_temp_source['Штабель'].astype(str)

# Извлекаем дату из 'Дата акта'
if 'Дата акта' in df_temp_source.columns and pd.api.types.is_datetime64_any_dtype(df_temp_source['Дата акта']):
    df_temp_source['Дата'] = df_temp_source['Дата акта'].dt.normalize()
    # Удаляем строки с некорректной датой, если они появились
    df_temp_source.dropna(subset=['Дата'], inplace=True)
else:
    print("Ошибка: Колонка 'Дата акта' отсутствует или не является datetime в данных температуры.")
    raise TypeError("Необходимая колонка 'Дата акта' не найдена или имеет неверный тип.")

# Убедимся, что 'Дата' в df_main тоже datetime (должна быть после parse_dates)
if not pd.api.types.is_datetime64_any_dtype(df_main['Дата']):
     df_main['Дата'] = pd.to_datetime(df_main['Дата'], errors='coerce')
     df_main.dropna(subset=['Дата'], inplace=True)

print("Ключи и типы подготовлены.")


# --- 11.3 Определение ключей по замерам температуры (Set 1) ---
print("\nОпределение ключей по дням замеров температуры...")
# Выбираем только уникальные комбинации Склад/Штабель/Дата из данных температуры
temp_keys_df = df_temp_source[['Склад', 'Штабель', 'Дата']].drop_duplicates()
print(f"Найдено уникальных ключей (Склад, Штабель, Дата) с замерами: {len(temp_keys_df)}")


# --- 11.4 Определение ключей по пожарам и 5 дням до (Set 2) ---
print("\nОпределение ключей по дням пожаров и 5 предшествующим дням...")
target_col = 'Fire_Started_Today'
days_before = 5

# Находим все события пожара
# fire_events_df = df_main[df_main[target_col] == 1][['Склад', 'Штабель', 'Дата']].copy()
# print(f"Найдено событий пожара: {len(fire_events_df)}")

# Собираем ключи в DataFrame для удобства
# fire_related_keys_list = []
# if not fire_events_df.empty:
#     for index, fire_event in tqdm(fire_events_df.iterrows(), total=len(fire_events_df), desc="Обработка пожаров"):
#         sklad = fire_event['Склад']
#         shtabel = fire_event['Штабель']
#         fire_date = fire_event['Дата']

#         # Добавляем ключи для дня пожара и 5 дней до
#         for k in range(days_before + 1): # от 0 до 5 включительно
#             target_date = fire_date - pd.Timedelta(days=k)
#             fire_related_keys_list.append({'Склад': sklad, 'Штабель': shtabel, 'Дата': target_date})

#     # Создаем DataFrame из списка и удаляем дубликаты
#     fire_keys_df = pd.DataFrame(fire_related_keys_list).drop_duplicates()
#     print(f"Найдено уникальных ключей, связанных с пожарами (+{days_before} дней до): {len(fire_keys_df)}")
# else:
#     print("Событий пожара не найдено, набор ключей пуст.")
#     fire_keys_df = pd.DataFrame(columns=['Склад', 'Штабель', 'Дата'])


# --- 11.5 Объединение ключей ---
print("\nОбъединение наборов ключей...")
# Объединяем два DataFrame с ключами
# combined_keys_df = pd.concat([temp_keys_df, fire_keys_df], ignore_index=True)
# Удаляем дубликаты, если день замера совпал с днем перед пожаром
# combined_keys_df.drop_duplicates(inplace=True)
# print(f"Общее количество уникальных ключей для сохранения: {len(combined_keys_df)}")


# --- 11.6 Фильтрация основного DataFrame ---
print("\nФильтрация основного DataFrame по объединенным ключам...")
# Используем inner merge для выполнения "полусоединения" (semi-join)
# Это оставит только те строки из df_main, ключи которых есть в combined_keys_df
# df_filtered = pd.merge(
#     df_main,
#     combined_keys_df,
#     on=['Склад', 'Штабель', 'Дата'],
#     how='inner' # Оставляем только совпадающие строки
# )

# print(f"\nРазмер основного DataFrame *до* фильтрации: {df_main.shape}")
# print(f"Размер основного DataFrame *после* фильтрации: {df_filtered.shape}")

# # --- 11.7 Результат ---
# print("\nФильтрация завершена.")
# # Переименовываем результат для ясности
# df_eda_filtered = df_filtered.copy()
print("\nПример отфильтрованных данных:")
print(df_eda_filtered.head())
print("\nПроверка целевой переменной в отфильтрованных данных:")
print(df_eda_filtered[target_col].value_counts())

# Опционально: Сохранение результата
save_filtered_df = True
if save_filtered_df:
    output_file_filtered = os.path.join(output_dir_final, 'master_grid_filtered_temp_fireperiod.csv')
    print(f"\nСохранение отфильтрованного DataFrame в {output_file_filtered}...")
    try:
        df_eda_filtered.to_csv(output_file_filtered, index=False)
        print("Файл успешно сохранен.")
    except Exception as e:
        print(f"Ошибка при сохранении файла: {e}")


print("\n--- Шаг 11 (Альтернативный) ЗАВЕРШЕН ---")
print("-" * 50)


--- Шаг 11 (Альтернативный): Фильтрация DataFrame по замерам температуры и периодам перед пожарами ---
Загрузка основного DataFrame из processed_data_final/master_grid_with_dynamic_features.csv...
Основной DataFrame загружен. Размер: (70176, 22)
Загрузка данных о температуре из processed_data_typed/temperature_A1_typed.csv...
Данные о температуре загружены. Размер: (4095, 5)

Подготовка ключей и типов данных...
Ключи и типы подготовлены.

Определение ключей по дням замеров температуры...
Найдено уникальных ключей (Склад, Штабель, Дата) с замерами: 2175

Определение ключей по дням пожаров и 5 предшествующим дням...

Объединение наборов ключей...

Фильтрация основного DataFrame по объединенным ключам...

Пример отфильтрованных данных:


NameError: name 'df_eda_filtered' is not defined

In [ ]:
!pip install catboost

In [ ]:
import pandas as pd
import os

# --- Шаг Y: Загрузка, Очистка и Сохранение CSV файла ---
print("\n--- Загрузка, Очистка и Сохранение CSV файла ---")

# 1. Укажи пути и имена файлов
input_directory = "."  # Укажи папку, где лежит файл (или "." для текущей)
input_filename = "master_grid_filtered_temp_fireperiod.csv" # Имя твоего файла
output_directory = "processed_data_final" # Папка для сохранения результата
output_filename = "master_grid_cleaned.csv" # Имя для очищенного файла

input_filepath = os.path.join(input_directory, input_filename)
output_filepath = os.path.join(output_directory, output_filename)

# 2. Список столбцов для удаления
cols_to_drop = [
    't_min',
    't_max',
    'p_min',
    'p_max',
    'humidity_min',
    'weather_code_mode'
    # Добавь сюда другие колонки, если нужно
]
print(f"Будут удалены столбцы (если они есть): {cols_to_drop}")

# 3. Проверка существования входного файла и загрузка
if os.path.exists(input_filepath):
    print(f"\nЗагрузка данных из файла: {input_filepath}...")
    try:
        # Загружаем DataFrame
        df_input = pd.read_csv(input_filepath)
        print(f"Файл успешно загружен. Размер: {df_input.shape}")
        print(f"Исходные колонки: {df_input.columns.tolist()}")

        # 4. Удаление столбцов
        # Находим столбцы, которые реально существуют
        existing_cols_to_drop = [col for col in cols_to_drop if col in df_input.columns]

        if existing_cols_to_drop:
            print(f"\nУдаление столбцов: {existing_cols_to_drop}...")
            try:
                # Удаляем столбцы
                df_cleaned = df_input.drop(columns=existing_cols_to_drop, errors='ignore')
                print("Столбцы успешно удалены.")
                print(f"Новое количество столбцов: {df_cleaned.shape[1]}")
                print(f"Оставшиеся колонки: {df_cleaned.columns.tolist()}")

                # 5. Сохранение результата в новый файл
                # Убедимся, что папка для вывода существует
                if not os.path.exists(output_directory):
                    os.makedirs(output_directory)
                    print(f"\nСоздана директория для вывода: {output_directory}")

                print(f"\nСохранение очищенного DataFrame в файл: {output_filepath}...")
                df_cleaned.to_csv(output_filepath, index=False)
                print("Файл успешно сохранен.")

            except Exception as e:
                print(f"Ошибка при удалении столбцов или сохранении файла: {e}")
        else:
            print("\nНи один из указанных столбцов для удаления не найден в DataFrame. Файл не изменен.")
            # Можно добавить сохранение исходного файла под новым именем, если нужно
            # print(f"\nКопирование исходного файла в: {output_filepath}...")
            # df_input.to_csv(output_filepath, index=False)
            # print("Файл скопирован без изменений.")

    except Exception as e:
        print(f"Ошибка при загрузке или обработке файла {input_filepath}: {e}")
else:
    print(f"Ошибка: Входной файл не найден по пути: {input_filepath}")

print("\n--- Обработка файла завершена ---")
print("-" * 50)